пошаговое руководство по реализации детекции оружия с помощью YOLOv4 из потокового видео

1. Установка необходимых библиотек
Установим OpenCV и Darknet (реализация YOLOv4):

In [ ]:
pip install opencv-python
git clone https://github.com/AlexeyAB/darknet
cd darknet
make

2. Настройка Darknet
Скачайте весы YOLOv4 и конфигурационные файлы:

In [ ]:
wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg
wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/data/coco.names
wget https://pjreddie.com/media/files/yolov4.weights

3. Код для детекции
Создайте скрипт для захвата видео и детекции объектов:

In [ ]:
import cv2
import numpy as np

# Загрузка модели YOLOv4
net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Захват видео с веб-камеры
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    height, width, channels = frame.shape

    # Препроцессинг изображения для YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Обработка результатов
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Применение non-max suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Отображение результатов на видео
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            if label == "person" or label == "gun":  # или другой класс, связанный с оружием
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Image', frame)
    key = cv2.waitKey(1)
    if key == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()

4. Запуск проекта
Запустите скрипт для начала детекции:

In [ ]:
python detect.py

Этот код захватывает потоковое видео с веб-камеры и использует YOLOv4 для детекции объектов, выводя результат в реальном времени.


5. Оптимизация производительности
Для улучшения производительности можно использовать GPU-ускорение с помощью CUDA. Если у вас есть поддерживаемая видеокарта, установите необходимые драйверы и библиотеку cuDNN, а затем соберите Darknet с поддержкой CUDA:

In [ ]:
# В Makefile установите USE_CUDA=1 и USE_CUDNN=1
make clean
make

6. Настройка классов детекции
Если необходимо детектировать конкретные классы, такие как оружие, вам нужно изменить файл coco.names, чтобы включить только те классы, которые вас интересуют. Если вы используете пользовательский датасет, измените файл конфигурации соответствующим образом.

7. Тренировка собственной модели
Если готовая модель YOLOv4 не распознает оружие должным образом, вам может потребоваться обучить собственную модель на специализированном датасете:

Соберите и разметьте данные (изображения с оружием).
Подготовьте файлы конфигурации для обучения (например, yolov4-custom.cfg).
Запустите обучение с использованием Darknet:

In [ ]:
./darknet detector train data/obj.data cfg/yolov4-custom.cfg yolov4.conv.137

8. Интеграция с другими системами
Для более сложных проектов можно интегрировать детекцию с системами видеонаблюдения, уведомлений или аналитики. Например, вы можете отправлять уведомления при обнаружении оружия:

In [ ]:
import smtplib
from email.mime.text import MIMEText

def send_alert():
    msg = MIMEText('Weapon detected!')
    msg['Subject'] = 'Alert'
    msg['From'] = 'your_email@gmail.com'
    msg['To'] = 'recipient_email@gmail.com'

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login('your_email@gmail.com', 'password')
    server.sendmail('your_email@gmail.com', 'recipient_email@gmail.com', msg.as_string())
    server.quit()

9. Развертывание на сервере
Если вам нужно развернуть систему на сервере:

Выберите сервер (например, AWS, Google Cloud, Azure).
Настройте сервер для запуска скрипта в фоновом режиме.
Настройте автоматический перезапуск скрипта при сбое (например, с помощью systemd).

In [ ]:
# /etc/systemd/system/detect.service
[Unit]
Description=YOLOv4 Weapon Detection

[Service]
ExecStart=/usr/bin/python3 /path/to/detect.py
Restart=always

[Install]
WantedBy=multi-user.target

In [ ]:
sudo systemctl enable detect.service
sudo systemctl start detect.service

10. Мониторинг и улучшение
Регулярно проверяйте работу системы и корректируйте модель по мере необходимости. Сохраняйте логи для анализа ошибок и доработок.

Таким образом, вы создадите и развернете систему детекции оружия с потокового видео, используя YOLOv4.

11. Интерфейс пользователя
Создайте простой веб-интерфейс для отображения видео и уведомлений. Используйте Flask для создания веб-приложения:

In [ ]:
pip install flask


In [ ]:
# app.py
from flask import Flask, render_template, Response
import cv2
import threading

app = Flask(__name__)
cap = cv2.VideoCapture(0)

def generate():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        height, width, channels = frame.shape

        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                if label == "person" or label == "gun":
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)


Создайте HTML шаблон для отображения видео:

In [ ]:
<!-- templates/index.html -->
<!DOCTYPE html>
<html>
<head>
    <title>Weapon Detection</title>
</head>
<body>
    <h1>Weapon Detection</h1>
    <img src="{{ url_for('video_feed') }}">
</body>
</html>


12. Запуск веб-приложения
Запустите Flask приложение:

In [ ]:
python app.py


Теперь вы можете получить доступ к веб-интерфейсу, открыв в браузере http://localhost:5000.

13. Обработка и хранение событий
Добавьте функцию для сохранения кадров с обнаруженным оружием и ведения журнала событий:

In [ ]:
import os
from datetime import datetime

def save_frame(frame):
    if not os.path.exists('events'):
        os.makedirs('events')
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    cv2.imwrite(f'events/{timestamp}.jpg', frame)

# Внутри цикла обработки видео:
if label == "gun":
    save_frame(frame)
    send_alert()


14. Тестирование и отладка
Тестируйте систему с различными видео и настройте параметры (порог уверенности, non-max suppression) для достижения наилучших результатов.

15. Обновление модели
Периодически обновляйте модель, добавляя новые данные и переобучая её для повышения точности детекции.

Следуя этим шагам, вы сможете создать и развернуть полноценную систему детекции оружия в реальном времени с веб-интерфейсом и уведомлениями.

16. Безопасность и авторизация
Для защиты веб-интерфейса добавьте авторизацию. Используем Flask-Login:

In [ ]:
pip install flask-login


Создайте файл для управления пользователями:

In [ ]:
# auth.py
from flask import Blueprint, render_template, redirect, url_for, request, flash
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

auth = Blueprint('auth', __name__)

users = {"admin": {"password": "password"}}  # Простая база данных пользователей

class User(UserMixin):
    def __init__(self, username):
        self.id = username

login_manager = LoginManager()
login_manager.login_view = 'auth.login'

@login_manager.user_loader
def load_user(user_id):
    if user_id in users:
        return User(user_id)
    return None

@auth.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if username in users and users[username]['password'] == password:
            user = User(username)
            login_user(user)
            return redirect(url_for('main.index'))
        flash('Invalid credentials')
    return render_template('login.html')

@auth.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('auth.login'))


17. Интеграция авторизации в основное приложение
Измените основной файл приложения для поддержки авторизации

In [ ]:
# app.py
from flask import Flask, render_template, Response
from flask_login import LoginManager, login_required
import cv2
import threading

app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.register_blueprint(auth, url_prefix='/auth')

login_manager.init_app(app)

# Ваша функция generate() и другие части кода остаются без изменений

@app.route('/')
@login_required
def index():
    return render_template('index.html')

@app.route('/video_feed')
@login_required
def video_feed():
    return Response(generate(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)


Создайте шаблон для страницы входа:

In [ ]:
<!-- templates/login.html -->
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
</head>
<body>
    <h1>Login</h1>
    <form method="POST">
        <label for="username">Username:</label>
        <input type="text" id="username" name="username" required>
        <br>
        <label for="password">Password:</label>
        <input type="password" id="password" name="password" required>
        <br>
        <button type="submit">Login</button>
    </form>
</body>
</html>


18. Логирование и мониторинг
Для обеспечения стабильности добавьте логирование и мониторинг:

In [ ]:
import logging

logging.basicConfig(filename='app.log', level=logging.INFO)

def save_frame(frame):
    if not os.path.exists('events'):
        os.makedirs('events')
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    cv2.imwrite(f'events/{timestamp}.jpg', frame)
    logging.info(f'Weapon detected and saved at {timestamp}')


19. Развертывание на сервере
Зарегистрируйтесь на платформе облачных вычислений (AWS, Google Cloud, Azure).
Создайте виртуальную машину (VM).
Установите необходимые зависимости и разверните приложение:

In [ ]:
# На вашем сервере
git clone <ваш_репозиторий>
cd <ваш_репозиторий>
pip install -r requirements.txt
python app.py


20. Настройка обратного прокси
Настройте Nginx или Apache для работы с вашим Flask приложением:

In [ ]:
# Установите Nginx
sudo apt update
sudo apt install nginx

# Настройте Nginx для проксирования запросов к вашему приложению
sudo nano /etc/nginx/sites-available/your_site

# Добавьте конфигурацию:
server {
    listen 80;
    server_name your_domain_or_IP;

    location / {
        proxy_pass http://127.0.0.1:5000;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
    }
}

# Включите конфигурацию и перезапустите Nginx
sudo ln -s /etc/nginx/sites-available/your_site /etc/nginx/sites-enabled
sudo systemctl restart nginx


Следуя этим шагам, вы создадите безопасную и функциональную систему детекции оружия с потокового видео, развернутую на сервере с веб-интерфейсом, авторизацией и логированием.

21. Масштабирование и балансировка нагрузки
Если ваш проект требует высокой доступности и масштабирования, настройте балансировщик нагрузки и несколько серверов приложений. Это поможет распределить нагрузку и обеспечить отказоустойчивость.

Балансировщик нагрузки: Используйте балансировщик нагрузки, такой как AWS ELB или Nginx, для распределения трафика между несколькими экземплярами вашего приложения.

Авто-масштабирование: Настройте авто-масштабирование на платформе облачных вычислений для автоматического добавления или удаления серверов в зависимости от нагрузки.

22. Контейнеризация с Docker
Использование Docker для контейнеризации вашего приложения упростит развертывание и управление зависимостями

Создайте Dockerfile:

In [ ]:
# Dockerfile
FROM python:3.8-slim

WORKDIR /app

COPY requirements.txt requirements.txt
RUN pip install -r requirements.txt

COPY . .

CMD ["python", "app.py"]


Создайте образ Docker:

In [ ]:
docker build -t weapon-detection:latest .


Запустите контейнер:

In [ ]:
docker run -d -p 5000:5000 weapon-detection:latest


Используйте Docker Compose для более сложных развертываний:

In [ ]:
# docker-compose.yml
version: '3'
services:
  web:
    image: weapon-detection:latest
    ports:
      - "5000:5000"
    environment:
      - FLASK_ENV=production


In [ ]:
docker-compose up -d


23. CI/CD для автоматизации развертываний
Настройте CI/CD (непрерывную интеграцию и доставку) для автоматизации развертываний.

Используйте GitHub Actions или другой CI/CD сервис:

In [ ]:
# .github/workflows/deploy.yml
name: Deploy

on:
  push:
    branches:
      - main

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout code
        uses: actions/checkout@v2
      - name: Set up Python
        uses: actions/setup-python@v2
        with:
          python-version: 3.8
      - name: Install dependencies
        run: pip install -r requirements.txt
      - name: Build Docker image
        run: docker build -t weapon-detection:latest .
      - name: Push Docker image
        run: |
          echo "${{ secrets.DOCKER_PASSWORD }}" | docker login -u "${{ secrets.DOCKER_USERNAME }}" --password-stdin
          docker tag weapon-detection:latest your_dockerhub_username/weapon-detection:latest
          docker push your_dockerhub_username/weapon-detection:latest
      - name: Deploy to server
        run: ssh user@your_server "docker pull your_dockerhub_username/weapon-detection:latest && docker-compose up -d"


Настройте сикреты в репозитории GitHub: добавьте DOCKER_USERNAME, DOCKER_PASSWORD, и другие необходимые переменные.

24. Мониторинг и алертинг
Для постоянного мониторинга системы и настройки уведомлений при сбоях используйте специализированные инструменты.

Prometheus и Grafana: для мониторинга метрик и создания дашбордов.

In [ ]:
docker run -d -p 9090:9090 --name prometheus prom/prometheus
docker run -d -p 3000:3000 --name grafana grafana/grafana


Alertmanager: для отправки уведомлений в случае проблем.

In [ ]:
docker run -d -p 9093:9093 --name alertmanager prom/alertmanager


Визуализация и отчетность
Создайте инструменты для визуализации данных и генерации отчетов, чтобы отслеживать эффективность системы и анализировать результаты детекции.

Grafana дашборды: Настройте дашборды для визуализации метрик.

Создайте панель мониторинга, показывающую количество детекций, типы обнаруженных объектов и другие важные метрики.
Используйте графики, гистограммы и таблицы для представления данных.
Автоматическая генерация отчетов: Используйте скрипты для создания регулярных отчетов о работе системы.

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

def generate_report():
    # Загрузка данных из логов или базы данных
    data = pd.read_csv('logs.csv')
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Фильтрация данных за последний месяц
    last_month = datetime.now() - pd.DateOffset(months=1)
    monthly_data = data[data['timestamp'] >= last_month]

    # Анализ данных
    report = monthly_data.groupby('label').size()

    # Визуализация
    plt.figure(figsize=(10, 5))
    report.plot(kind='bar')
    plt.title('Detections in the Last Month')
    plt.xlabel('Object Type')
    plt.ylabel('Count')
    plt.savefig('report.png')
    plt.close()

    # Сохранение отчета
    report.to_csv('monthly_report.csv')


Обратная связь и улучшения
Собирайте обратную связь от пользователей и регулярно вносите улучшения в систему.

Форма обратной связи: Создайте страницу для пользователей, чтобы они могли сообщать о проблемах и предлагать улучшения.

In [ ]:
<!-- templates/feedback.html -->
<!DOCTYPE html>
<html>
<head>
    <title>Feedback</title>
</head>
<body>
    <h1>Feedback</h1>
    <form action="/feedback" method="POST">
        <label for="feedback">Your Feedback:</label>
        <textarea id="feedback" name="feedback" required></textarea>
        <br>
        <button type="submit">Submit</button>
    </form>
</body>
</html>


Обработка обратной связи: Добавьте маршрут в Flask для обработки формы обратной связи.

In [ ]:
# app.py
from flask import request, redirect, url_for, flash

@app.route('/feedback', methods=['POST'])
def feedback():
    feedback_text = request.form['feedback']
    with open('feedback.txt', 'a') as f:
        f.write(f'{datetime.now()}: {feedback_text}\n')
    flash('Thank you for your feedback!')
    return redirect(url_for('index'))


Обеспечение качества и тестирование
Регулярное тестирование и обеспечение качества системы помогут поддерживать её работоспособность и надежность.

Юнит-тесты: Пишите тесты для отдельных частей кода.

In [ ]:
import unittest

class TestDetection(unittest.TestCase):
    def test_save_frame(self):
        # Тестирование функции сохранения кадра
        frame = np.zeros((100, 100, 3), dtype=np.uint8)
        save_frame(frame)
        self.assertTrue(os.path.exists('events'))

if __name__ == '__main__':
    unittest.main()


Интеграционные тесты: Тестируйте всю систему целиком.

In [ ]:
from selenium import webdriver

def test_web_interface():
    driver = webdriver.Chrome()
    driver.get('http://localhost:5000')
    assert 'Weapon Detection' in driver.title
    driver.quit()


Машинное обучение и улучшение моделей
Используйте передовые методы машинного обучения для улучшения моделей.

Transfer Learning: Используйте предварительно обученные модели и дообучайте их на своем датасете.

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=5, validation_data=(val_data, val_labels))


Data Augmentation: Используйте методы аугментации данных для увеличения разнообразия обучающего набора данных.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(train_data)
